# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine


import re

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.metrics import accuracy_score, precision_score, recall_score,f1_score

In [13]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/abdullahghifari/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/abdullahghifari/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/abdullahghifari/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/abdullahghifari/nltk_data...


True

In [14]:
# load data from database
engine = create_engine('sqlite:///disaster.db')
df = pd.read_sql_table('messages_clean_table',con=engine)

In [15]:
X = df['message']
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [16]:
def tokenize(text):
    
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    words = word_tokenize(text)
    words = [w for w in words if w not in stopwords.words("english")]
    
    words = [PorterStemmer().stem(w) for w in words]
    words = [WordNetLemmatizer().lemmatize(w) for w in words]
    
    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [17]:
vectorizer = TfidfVectorizer( tokenizer=tokenize ,use_idf=True, smooth_idf=True, sublinear_tf=False)

rf = RandomForestClassifier(random_state=123)

pipeline = Pipeline([
    ('vectorizer',vectorizer),
    ('model',rf)
])# compute bag of word counts and tf-idf values

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [18]:
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.3,random_state=123)
pipeline.fit(X_train,y_train)

/Users/abdullahghifari/anaconda3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x15da1eb90>)),
                ('model', RandomForestClassifier(random_state=123))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [19]:
y_train_pred = pipeline.predict(X_train)
y_test_pred = pipeline.predict(X_test)

In [20]:
def get_score(y_true,y_pred):
    """Get Score for Each Column"""
    
    accuracy = accuracy_score(y_true,y_pred)
    precision =round( precision_score(y_true,y_pred,average='micro'))
    recall = recall_score(y_true,y_pred,average='micro')
    f1 = f1_score(y_true,y_pred,average='micro')
    
    return {'Precision':precision, 'Recall':recall,'F1-score':f1,'Accuracy':accuracy}

In [21]:
train_results = []
for i,column in enumerate(y_train.columns):
    result = get_score(y_train.loc[:,column].values,y_train_pred[:,i])
    train_results.append(result)
train_results_df = pd.DataFrame(train_results)
train_results_df    

,Precision,Recall,F1-score,Accuracy
0,1,0.998311,0.998311,0.998311
1,1,0.999401,0.999401,0.999401
2,1,1.000000,1.000000,1.000000
3,1,0.999019,0.999019,0.999019
4,1,0.999564,0.999564,0.999564
5,1,0.999728,0.999728,0.999728
6,1,0.999837,0.999837,0.999837
7,1,0.999673,0.999673,0.999673
8,1,0.999891,0.999891,0.999891
9,1,1.000000,1.000000,1.000000


In [22]:
test_results = []
for i,column in enumerate(y_test.columns):
    result = get_score(y_test.loc[:,column].values,y_test_pred[:,i])
    test_results.append(result)
test_results_df = pd.DataFrame(test_results)
test_results_df

,Precision,Recall,F1-score,Accuracy
0,1,0.827845,0.827845,0.827845
1,1,0.903751,0.903751,0.903751
2,1,0.995677,0.995677,0.995677
3,1,0.767324,0.767324,0.767324
4,1,0.919898,0.919898,0.919898
5,1,0.950286,0.950286,0.950286
6,1,0.975461,0.975461,0.975461
7,1,0.982835,0.982835,0.982835
8,1,0.967196,0.967196,0.967196
9,1,1.000000,1.000000,1.000000


In [23]:
train_results_df['F1-score'].mean()

0.9997366174413744

In [24]:
test_results_df['F1-score'].mean()

0.9463975418520872

### 6. Improve your model
Use grid search to find better parameters. 

In [29]:
def custom_multiclf_score(y_true,y_pred):
    list_results = []
    for i,column in enumerate(y_true.columns):
        result = f1_score(y_true.loc[:,column].values,y_pred[:,i],average='micro')
        list_results.append(result)
    list_results = np.array(list_results)
    return list_results.mean()

In [30]:
from sklearn.metrics import make_scorer
score = make_scorer(custom_multiclf_score)

In [32]:
parameters = {
    'model__max_depth':[2,4,None],
    'model__max_features':['sqrt','log2',None],
    'model__criterion':['gini','entropy'],
    'model__min_samples_leaf':[1,2,3]
}

cv = GridSearchCV(
    estimator=pipeline,
    param_grid=parameters,
    scoring=score,
#     n_jobs=-1, 
    cv=5, 
    refit=True, 
    return_train_score=True,
    verbose=10
)

In [ ]:
cv.fit(X_train,y_train)

Fitting 5 folds for each of 54 candidates, totalling 270 fits
[CV 1/5; 1/54] START model__criterion=gini, model__max_depth=2, model__max_features=sqrt, model__min_samples_leaf=1


/Users/abdullahghifari/anaconda3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 1/5; 1/54] END model__criterion=gini, model__max_depth=2, model__max_features=sqrt, model__min_samples_leaf=1;, score=(train=0.926, test=0.927) total time=  27.0s
[CV 2/5; 1/54] START model__criterion=gini, model__max_depth=2, model__max_features=sqrt, model__min_samples_leaf=1


/Users/abdullahghifari/anaconda3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 2/5; 1/54] END model__criterion=gini, model__max_depth=2, model__max_features=sqrt, model__min_samples_leaf=1;, score=(train=0.927, test=0.925) total time=  27.6s
[CV 3/5; 1/54] START model__criterion=gini, model__max_depth=2, model__max_features=sqrt, model__min_samples_leaf=1


...

/Users/abdullahghifari/anaconda3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 3/5; 25/54] END model__criterion=gini, model__max_depth=None, model__max_features=None, model__min_samples_leaf=1;, score=(train=1.000, test=0.947) total time=22.5min
[CV 4/5; 25/54] START model__criterion=gini, model__max_depth=None, model__max_features=None, model__min_samples_leaf=1


/Users/abdullahghifari/anaconda3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 4/5; 25/54] END model__criterion=gini, model__max_depth=None, model__max_features=None, model__min_samples_leaf=1;, score=(train=1.000, test=0.946) total time=10.9min
[CV 5/5; 25/54] START model__criterion=gini, model__max_depth=None, model__max_features=None, model__min_samples_leaf=1


/Users/abdullahghifari/anaconda3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_train_pred = cv.predict(X_train)
y_test_pred = cv.predict(X_test)

In [ ]:
train_results = []
for i,column in enumerate(y_train.columns):
    result = get_score(y_train.loc[:,column].values,y_train_pred[:,i])
    train_results.append(result)
train_results_df = pd.DataFrame(train_results)
train_results_df    

In [ ]:
test_results = []
for i,column in enumerate(y_test.columns):
    result = get_score(y_test.loc[:,column].values,y_test_pred[:,i])
    test_results.append(result)
test_results_df = pd.DataFrame(test_results)
test_results_df

In [ ]:
train_results_df['F1-score'].mean()

In [ ]:
test_results_df['F1-score'].mean()

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
vectorizer = CountVectorizer( tokenizer=tokenize ,use_idf=True, smooth_idf=True, sublinear_tf=False)

model = MultinomialNB(random_state=123)

pipeline = Pipeline([
    ('vectorizer',vectorizer),
    ('model',model)
])

In [ ]:
parameters = {
    'clf__force_alpha':[True,False]
}

In [ ]:
cv_nb = GridSearchCV(
    pipeline,
    param_grid=parameters, 
    scoring=score,
    n_jobs = -1, 
    cv = 5, 
    refit = True, 
    return_train_score=True,
    verbose=10)

cv_nb.fit(X_train, y_train)

In [ ]:
y_train_pred = cv.predict(X_train)
y_test_pred = cv.predict(X_test)

In [ ]:
train_results = []
for i,column in enumerate(y_train.columns):
    result = get_score(y_train.loc[:,column].values,y_train_pred[:,i])
    train_results.append(result)
train_results_df = pd.DataFrame(train_results)
train_results_df    

In [ ]:
test_results = []
for i,column in enumerate(y_test.columns):
    result = get_score(y_test.loc[:,column].values,y_test_pred[:,i])
    test_results.append(result)
test_results_df = pd.DataFrame(test_results)
test_results_df

In [ ]:
train_results_df['F1-score'].mean()

In [ ]:
test_results_df['F1-score'].mean()

### 9. Export your model as a pickle file

In [ ]:
import pickle

filename = 'best_model_rf_tuned.pkl'
pickle.dump(cv, open(filename, 'wb'))

filename = 'best_model_nb.pkl'
pickle.dump(cv_nb, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.